# - Data Warehouse -

### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [ ]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [ ]:
Workorderdf = run_query("SELECT * FROM Production.Workorder", "AdventureWorks2019")
WorkOrderRoutingdf = run_query("SELECT * FROM Production.WorkOrderRouting", "AdventureWorks2019")

Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

In [ ]:
merged_df = pd.merge(Workorderdf, WorkOrderRoutingdf, on='WorkOrderID', how='inner')

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [ ]:
# SQL query
WorkOrder_sql = """
CREATE TABLE WorkOrder (
    WorkOrderID INT PRIMARY KEY,
    ProductID INT,
    OrderQty INT,
    StockedQty INT,
    ScrappedQty INT,
    StartDate DATETIME,
    EndDate DATETIME,
    DueDate DATETIME,
    ScrapReasonID INT,
    ModifiedDate DATETIME,
    OperationSequence INT,
    LocationID INT,
    ScheduledStartDate DATETIME,
    ScheduledEndDate DATETIME,
    ActualStartDate DATETIME,
    ActualEndDate DATETIME,
    ActualResourceHrs DECIMAL(10,2),
    PlannedCost DECIMAL(10,2),
    ActualCost DECIMAL(10,2)
);
"""

db_name = "MustafaTest"
conn, cursor = create_connection(db_name)
cursor.execute(WorkOrder_sql)
conn.commit()
conn.close()

#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [ ]:
# Verbinding maken met de database
test_database_name = 'MustafaTest'
conn, cursor = create_connection(test_database_name)

for index, row in merged_df.iterrows():
    # Opstellen van de SQL-invoegquery
    query = f"""
        INSERT INTO WorkOrder (
            WorkOrderID, ProductID, OrderQty, StockedQty, ScrappedQty, StartDate, EndDate, DueDate, 
            ScrapReasonID, ModifiedDate, OperationSequence, LocationID, ScheduledStartDate, 
            ScheduledEndDate, ActualStartDate, ActualEndDate, ActualResourceHrs, PlannedCost, ActualCost
        ) 
        VALUES (
            {row['WorkOrderID']}, {row['ProductID']}, {row['OrderQty']}, {row['StockedQty']}, {row['ScrappedQty']}, 
            {row['StartDate']}, {row['EndDate']}, {row['DueDate']}, {row['ScrapReasonID']}, {row['ModifiedDate']}, 
            {row['OperationSequence']}, {row['LocationID']}, {row['ScheduledStartDate']}, {row['ScheduledEndDate']}, 
            {row['ActualStartDate']}, {row['ActualEndDate']}, {row['ActualResourceHrs']}, {row['PlannedCost']}, 
            {row['ActualCost']}
        )
    """
    # Uitvoeren van de query
    cursor.execute(query)

conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [ ]:
# Controleren op ontbrekende waarden
missing_values = merged_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = merged_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)

**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.